In [ ]:
from __future__ import division
import numpy as np
import os
import glob

from random import *
from PIL import Image
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.image as mpimg

from keras.backend import squeeze
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU, Activation, BatchNormalization, Bidirectional
from keras.layers.convolutional import Convolution2D, Cropping2D, ZeroPadding2D, MaxPooling2D
from keras.layers.recurrent import LSTM, StackedRNNCells
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam, RMSprop
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
d = {}
from subprocess import check_output
with open('gdrive/My Drive/Words.txt') as f:
    for line in f:
        imagename = line.split(' ')[0]
        wordInImage = line.split(' ')[1]
        d[imagename] = wordInImage

In [ ]:
tmp = []
target_list = []

path_to_files = os.path.join('gdrive/My Drive/word test', '*')
for filename in sorted(glob.glob(path_to_files)):
  tmp.append(filename)
  image_name = filename.split('/')[-1]
  file, ext = os.path.splitext(image_name)
  parts = file.split('.')
  word = parts[0]
  for key in d:
    if key == word:
      target_list.append(str(d[word]))

img_files = np.asarray(tmp)
img_targets = np.asarray(target_list)

In [ ]:
train_files, rem_files, train_targets, rem_targets = train_test_split(
        img_files, img_targets, train_size=0.66, random_state=52, shuffle= True)

validation_files, test_files, validation_targets, test_targets = train_test_split(
        rem_files, rem_targets, train_size=0.5, random_state=22, shuffle=True)

print(train_files.shape, validation_files.shape, test_files.shape)
print(train_targets.shape, validation_targets.shape, test_targets.shape)

In [ ]:
batch_size = 20
num_classes = 50

def generate_data(samples, target_files,  batch_size=batch_size, factor = 0.1 ):
  num_samples = len(samples)
  from sklearn.utils import shuffle
  while 1: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            batch_targets = target_files[offset:offset+batch_size]

            images = []
            targets = []
            for i in range(len(batch_samples)):
                batch_sample = batch_samples[i]
                batch_target = batch_targets[i]
                im = Image.open(batch_sample)
                cur_width = im.size[0]
                cur_height = im.size[1]

                # print(cur_width, cur_height)
                height_fac = 113 / cur_height

                new_width = int(cur_width * height_fac)
                size = new_width, 113

                imresize = im.resize((size), Image.ANTIALIAS)  # Resize so height = 113 while keeping aspect ratio
                now_width = imresize.size[0]
                now_height = imresize.size[1]
                # Generate crops of size 113x113 from this resized image and keep random 10% of crops

                avail_x_points = list(range(0, now_width - 113 ))# total x start points are from 0 to width -113

                # Pick random x%
                pick_num = int(len(avail_x_points)*factor)

                # Now pick
                random_startx = sample(avail_x_points,  pick_num)

                for start in random_startx:
                    imcrop = imresize.crop((start, 0, start+113, 113))
                    images.append(np.asarray(imcrop))
                    targets.append(batch_target)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(targets)

            #reshape X_train for feeding in later
            X_train = X_train.reshape(X_train.shape[0], 113, 113, 1)
            #convert to float and normalize
            X_train = X_train.astype('float32')
            X_train /= 255

            #One hot encode y
            y_train = to_categorical(y_train, num_classes)
            yield shuffle(X_train, y_train)


In [ ]:
train_generator = generate_data(train_files, train_targets, batch_size=batch_size, factor = 0.3)
validation_generator = generate_data(validation_files, validation_targets, batch_size=batch_size, factor = 0.3)
test_generator = generate_data(test_files, test_targets, batch_size=batch_size, factor = 0.1)

In [ ]:
def resize_image(image):
    import tensorflow as tf
    return tf.image.resize(image,[128,32])

In [ ]:
# Function to resize image to 64x64
row, col, ch = 113, 113, 1

model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(row, col, ch)))

# Resise data within the neural network
model.add(Lambda(resize_image))  #resize images to allow for easy computation
#model.add(Lambda(lambda x: resize_image))

# CNN model - Building the model suggested in paper

model.add(Convolution2D(filters= 32, kernel_size =(5,5), strides= (2,2), padding='same', name='conv1')) #96
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2), name='pool1'))

model.add(Convolution2D(filters= 64, kernel_size =(3,3), strides= (1,1), padding='same', name='conv2'))  #256
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2), name='pool2'))

model.add(Convolution2D(filters=128, kernel_size =(3,3), strides= (1,1), padding='same', name='conv3'))  #256
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2), name='pool3'))

model.add(Bidirectional(LSTM('relu', dropout=0.5, return_sequences=True, time_major=False)))

model.add(Dense(128, name='dense1'))  #1024
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(64, name='dense2'))  #1024
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes,name='output'))
model.add(Activation('softmax'))  #softmax since output is within 50 classes
print("Worked till here")

model.compile(loss='ctc_beam_search_decoder', optimizer=Adam(), metrics=['accuracy'])

print(model.summary())

In [ ]:
# Training the Model
nb_epoch = 4

samples_per_epoch = 658
nb_val_samples = 170


# #save every model using Keras checkpoint
from keras.callbacks import ModelCheckpoint
filepath="gdrive/My Drive/check-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath= filepath, verbose=1, save_best_only=False)
callbacks_list = [checkpoint]



In [ ]:
# #Model fit generator
model.fit_generator(train_generator, steps_per_epoch= samples_per_epoch,
                                      validation_data=validation_generator,
                                      validation_steps=nb_val_samples, epochs=nb_epoch, verbose=1, callbacks=callbacks_list)
model.load_weights('drive/My Drive/check-04-6.6100.hdf5')#load the checkpoint whatever created, it will show at the end after completion of model.fit()
scores = model.evaluate_generator(test_generator,245) 
print("Accuracy = ", scores[1]*100)

In [ ]:
 images = []
for filename in test_files[:50]:
  im = Image.open(filename)
  cur_width = im.size[0]
  cur_height = im.size[1]
  #print("Before Crop:", cur_width, cur_height)
  height_fac = 113 / cur_height
  new_width = int(cur_width * height_fac)
  size = new_width, 113
  imresize = im.resize((size), Image.ANTIALIAS)  # Resize so height = 113 while keeping aspect ratio
  now_width = imresize.size[0]
  now_height = imresize.size[1]
  #print("After Crop:", now_width, now_height)
  avail_x_points = list(range(0, now_width - 113 ))# total x start points are from 0 to width -113
  # Pick random x%
  factor = 0.1
  pick_num = int(len(avail_x_points)*factor)
  #print("Pick Number is ::", pick_num)
  random_startx = sample(avail_x_points,  pick_num)
  for start in random_startx:
    imcrop = imresize.crop((start, 0, start+113, 113))
    images.append(np.asarray(imcrop))
  X_test = np.array(images)
  X_test = X_test.reshape(X_test.shape[0], 113, 113, 1)
  #convert to float and normalize
  X_test = X_test.astype('float32')
  X_test /= 255
  shuffle(X_test)
print(X_test.shape)

predictions = model.predict(X_test,verbose=1)
print(predictions.shape)
predicted_word = []
for pred in predictions:
     predicted_word.append(np.argmax(pred))
print(len(predicted_word))